[Day 2: 1202 Program Alarm](https://adventofcode.com/2019/day/2)

In [1]:
import numpy as np
from itertools import product

In [2]:
with open('../input/D02.txt', 'r') as f:
    codes = [int(c) for c in f.read().split(',')]

In [3]:
def intcode_computer(codes, pointer=0, input=None):
    output = []
    relative_base = 0
    def get_value(mode, value):
        if mode == 0: return codes[value]
        elif mode == 1: return value
        elif mode == 2: return codes[value] + relative_base
    while True:
        instruction = str(codes[pointer]).zfill(5)
        code, mode = int(instruction[-2:]), [int(d) for d in instruction[-3::-1]]
        if code == 99 or (code == 3 and not input):
            return code, output, pointer
        elif code == 1:
            codes[get_value(mode[2], pointer+3)] = codes[get_value(mode[0], pointer+1)]+codes[get_value(mode[1], pointer+2)]
            pointer += 4
        elif code == 2:
            codes[get_value(mode[2], pointer+3)] = codes[get_value(mode[0], pointer+1)]*codes[get_value(mode[1], pointer+2)]
            pointer += 4
        elif code == 3:
            codes[get_value(mode[0], pointer+1)] = input.pop(0)
            pointer += 2
        elif code == 4:
            output.append(codes[get_value(mode[0], pointer+1)])
            pointer += 2
        elif code == 5:
            pointer = codes[get_value(mode[1], pointer+2)] if codes[get_value(mode[0], pointer+1)] else pointer+3
        elif code == 6:
            pointer = pointer+3 if codes[get_value(mode[0], pointer+1)] else codes[get_value(mode[1], pointer+2)]
        elif code == 7:
            codes[get_value(mode[2], pointer+3)] = 1 if codes[get_value(mode[0], pointer+1)] < codes[get_value(mode[1], pointer+2)] else 0
            pointer += 4
        elif code == 8:
            codes[get_value(mode[2], pointer+3)] = 1 if codes[get_value(mode[0], pointer+1)] == codes[get_value(mode[1], pointer+2)] else 0
            pointer += 4
        elif code == 9:
            relative_base += (codes[get_value(mode[0], pointer+1)])
            pointer += 2

In [4]:
noun = 12
verb = 2
def set_noun_verb(codes, n, v):
    data = codes.copy()
    data[1] = n
    data[2] = v
    intcode_computer(data)
    return data[0]
set_noun_verb(codes.copy(), noun, verb)

2692315

In [5]:
def find_noun_verb(codes, value):
    for n, v in product(range(100), range(100)):
        if set_noun_verb(codes, n, v) == value:
            return 100*n+v
    return None
find_noun_verb(codes.copy(), 19690720)

9507

[Day 5: Sunny with a Chance of Asteroids](https://adventofcode.com/2019/day/5)

In [6]:
with open('../input/D05.txt', 'r') as f:
    codes = [int(c) for c in f.read().split(',')]

In [7]:
intcode_computer(codes.copy(), input=[1])[1]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 6069343]

In [8]:
intcode_computer(codes.copy(), input=[5])[1]

[3188550]

[Day 7: Amplification Circuit](https://adventofcode.com/2019/day/7)

In [9]:
with open('../input/D07.txt', 'r') as f:
    codes = [int(c) for c in f.read().split(',')]

In [10]:
from itertools import permutations

In [11]:
def get_highest_signal(codes, settings, feedback=False):
    highest_signal = 0
    final_setting = None
    for setting in permutations(settings):
        amplifier = 0
        codes_setting = dict((setup, codes.copy()) for setup in setting)
        pointer_setting = dict((setup, 0) for setup in setting)
        input_setting = dict((setup, [setup]) for setup in setting)
        input_setting[setting[0]].append(0)
        while amplifier<len(setting) or feedback:
            setup = setting[amplifier % len(setting)]
            opcode, output, pointer_setting[setup] = intcode_computer(codes_setting[setup], pointer=pointer_setting[setup], input=input_setting[setup]) # My Code
            next_setup = setting[(amplifier+1) % len(setting)]
            input_setting[next_setup] += output
            if opcode == 99 and not (amplifier + 1) % len(setting):
                break
            amplifier += 1
        if output[0]>highest_signal:
            highest_signal = output[0]
            final_setting = setting
    return highest_signal, final_setting

In [12]:
get_highest_signal(codes.copy(), range(5), feedback=False)

(75228, (0, 3, 4, 2, 1))

In [13]:
get_highest_signal(codes.copy(), range(5, 10), feedback=True)

(79846026, (6, 7, 9, 5, 8))

[Day 9: Sensor Boost](https://adventofcode.com/2019/day/9)

In [14]:
from collections import defaultdict
with open('../input/D09.txt', 'r') as f:
    codes = [int(c) for c in f.read().split(',')]
    codes = defaultdict(lambda: 0, zip(range(len(codes)), codes))

In [15]:
intcode_computer(codes.copy(), input=[1])[1]

[3839402290]

In [16]:
intcode_computer(codes.copy(), input=[2])[1]

[35734]

Taken from **Dor Kedem** code

In [17]:
from operator import add, mul, lt, eq
from copy import copy

def exec_intcode_program(program, op_pointer=0, inputs=None):
    """ Executes the intcode program, used in days 2, 5, 7 & 9 of Advent of Code 2019. 
        - program: The program to execute.
        - op_pointer: The pointer to the op to start the program at.
        - inputs: The set of inputs for the program when it is requested. 
        The program will be adjusted, and will stop when requested to (op=99),
        or when requested for input (op=3) without available input.
        """
    # Define the relative base of the operations.
    relative_base = 0
    def change_relative_base(change): 
        nonlocal relative_base 
        relative_base += change
    # Define the program operations.
    opcode_details = {
        1: {"in_params_count": 2, "store_output": 1, "op": add}, 
        2: {"in_params_count": 2, "store_output": 1, "op": mul}, 
        3: {"in_params_count": 0, "store_output": 1, "op": lambda: inputs.pop(0)}, 
        4: {"in_params_count": 1, "store_output": 0, "op": lambda x: outputs.append(x)},
        5: {"in_params_count": 2, "store_output": 0, "op": lambda cond, addr: addr if cond else None},
        6: {"in_params_count": 2, "store_output": 0, "op": lambda cond, addr: None if cond else addr},
        7: {"in_params_count": 2, "store_output": 1, "op": lt},
        8: {"in_params_count": 2, "store_output": 1, "op": eq},
        9: {"in_params_count": 1, "store_output": 0, "op": change_relative_base}
    }
    # Define the logic to converting parameters to their values' addresses from the param mode.
    def get_param_value_address(param_address, param_mode):
        if   param_mode == 0: return program[param_address]
        elif param_mode == 1: return param_address
        elif param_mode == 2: return program[param_address] + relative_base
    # Start the program.
    outputs = []
    while True:
        # Break down op pointer to op instruction parts.
        op_instruction = str(program[op_pointer]).zfill(5)
        opcode, param_modes = int(op_instruction[-2:]), [int(digit) for digit in op_instruction[-3::-1]]
        # Make sure we don't need to stop the program.
        if opcode == 99 or (opcode == 3 and not inputs):
            return opcode, op_pointer, outputs 
        # Retrieve opcode details from opcode.
        in_params_count = opcode_details[opcode]["in_params_count"]
        store_output    = opcode_details[opcode]["store_output"]
        op              = opcode_details[opcode]["op"]
        # Retrieve parameter value address from the param mode.
        in_params_values = [program[get_param_value_address(param_address, param_mode)] for param_address, param_mode
                            in zip(range(op_pointer+1, op_pointer+1+in_params_count), param_modes[:in_params_count])]
        # Execute the operation.
        out_val = op(*in_params_values)
        # Store the output in the out_param specified address if required.
        if store_output:
            out_address = get_param_value_address(param_address=op_pointer + 1 + in_params_count,
                                                  param_mode=param_modes[in_params_count])
            program[out_address] = out_val
        # Advance the op pointer (jump if successful jump command, advance otherwise)
        op_pointer = (out_val if (opcode in [5,6] and out_val is not None) 
                      else op_pointer + 1 + in_params_count + store_output)